In [1]:
import pandas as pd
from tqdm import tqdm
import arxiv
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("nips2019.csv")

In [3]:
df[:10] # not every accepted papers in NIPS 2019 available on arxiv today

,title,authors,arxiv_id
0,Blind Super-Resolution Kernel Estimation using...,Sefi Bell-Kligler;Assaf Shocher;Michal Irani,NaN
1,Guided Similarity Separation for Image Retrieval,Chundi Liu;Guangwei Yu;Maksims Volkovs;Cheng C...,NaN
2,"Average Individual Fairness: Algorithms, Gener...",Saeed Sharifi-Malvajerdi;Michael Kearns;Aaron ...,http://arxiv.org/abs/1905.10607v1
3,Greedy InfoMax for Biologically Plausible Self...,Sindy Löwe;Peter O'Connor;Bastiaan Veeling,NaN
4,Dynamics of stochastic gradient descent for tw...,Sebastian Goldt;Madhu Advani;Andrew Saxe;Flore...,NaN
5,Parameter elimination in particle Gibbs sampling,Anna Wigren;Riccardo Sven Risuleo;Lawrence Mur...,NaN
6,Nonparametric Density Estimation & Convergence...,Ananya Uppal;Shashank Singh;Barnabas Poczos,NaN
7,On Robustness of Principal Component Regression,Anish Agarwal;Devavrat Shah;Dennis Shen;Dogyoo...,NaN
8,Scalable Bayesian inference of dendritic volta...,Ruoxi Sun; Ian Kinsella;Scott Linderman;Liam ...,NaN
9,Optimizing Generalized Rate Metrics through Th...,Harikrishna Narasimhan;Andrew Cotter;Maya Gupta,NaN


In [4]:
print("Only {:.2f} % of NIPS 2019 papers available on arxiv by {}".format(len(df.dropna())/len(df)*100, datetime.now()))

Only 16.57 % of NIPS 2019 papers available on arxiv by 2019-09-18 07:40:17.329050


In [5]:
def retrieve_arxiv(title, authors, k=5):
    query = "title:{} au:{}".format(title, authors[0].lower().replace(" ", "_"))
    results = arxiv.query(query, max_results=k)
    found = False
    record = None
    for result in results:
        if title.strip().lower() == result["title"].strip().lower():
            if set(authors) == set(result["authors"]):
                found = True
                record = result
                break
    return record, found, results

In [ ]:
for i in tqdm(range(len(df))):
    if not isinstance(df.loc[i]["arxiv_id"], str):
        title = df.loc[i]["title"]
        authors = df.loc[i]["authors"].split(";")
        record, results, found = retrieve_arxiv(title, authors)
        if record is not None and found:
            df.loc[i]["arxiv_id"] = record["id"]

 97%|████████████████████████████████████████████████▌ | 1388/1430 [29:35<00:53,  1.28s/it]

In [ ]:
df.to_csv("nips2019.csv", index=False) # a little bit increased
print("Only {:.2f} % of NIPS 2019 papers available on arxiv by {}".format(len(df.dropna())/len(df)*100, datetime.now()))

In [ ]:
df.dropna()[:10]